In [24]:
# INSTALAR pip install python-docx / pip install openpyxl 
import os
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_LINE_SPACING
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.oxml import parse_xml

# === CONFIGURACIÓN ===
carpeta_fna = r"C:\Tareas002\SegundoParcial\genomas_ecoli"  
archivo_blatem1 = r"C:\Tareas002\SegundoParcial\Gen_blaTEM-1_eje1.fasta"
archivo_blactxm15 = r"C:\Tareas002\SegundoParcial\Gen_blaCTX-M-15_eje1.fasta"
salida_docx = r"C:\Tareas002\SegundoParcial\Informes"  # Carpeta para informes DOCX
salida_excel = r"C:\Tareas002\SegundoParcial\Informes\resumen_resultados.xlsx"  # Archivo Excel de resumen

# === FUNCIONES DE APOYO ===
def leer_gen(ruta):
    """Lee un archivo FASTA y devuelve la secuencia sin saltos de línea"""
    with open(ruta, "r") as f:
        return f.read().replace("\n", "").upper()

def reverse_complement(sequence):
    """Devuelve el reverso complemento de una secuencia de ADN"""
    return str(Seq(sequence).reverse_complement())

def add_horizontal_line(doc):
    """Añade una línea horizontal al documento"""
    paragraph = doc.add_paragraph()
    p = paragraph._p
    pPr = p.get_or_add_pPr()
    pBdr = OxmlElement('w:pBdr')
    pPr.insert_element_before(pBdr, 
        *[t for t in pPr.iterchildren() if t.tag.endswith(('ind', 'jc'))])
    bottom = OxmlElement('w:bottom')
    bottom.set(qn('w:val'), 'single')
    bottom.set(qn('w:sz'), '12')
    bottom.set(qn('w:space'), '1')
    bottom.set(qn('w:color'), '000000')
    pBdr.append(bottom)

def add_run(paragraph, text, bold=False, underline=False, highlight=False):
    
    run = paragraph.add_run(text)
    run.font.name = 'Tahoma'
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Tahoma')
    run.font.size = Pt(10)
    run.bold = bold
    run.underline = underline
    

def generar_informe(nombre_archivo, resultado_blatem1, resultado_blactxm15):
    
    bacteria_nombre = nombre_archivo.replace(".fna", "")
    doc = Document()

    # Configuración de márgenes
    section = doc.sections[0]
    section.top_margin = Inches(1)
    section.bottom_margin = Inches(1)
    section.left_margin = Inches(1.25)
    section.right_margin = Inches(1.25)
    
    # Título
    titulo = doc.add_paragraph()
    titulo.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = titulo.add_run("MODELO DE INFORME DE CORRELACIÓN\nCLÍNICA ADMINISTRATIVA DEL SUS")
    run.bold = True

    # Tabla de datos administrativos
    ancho_total_in = 8.27 - 1.25 - 1.25
    ancho_col0 = 0.75
    ancho_col1 = ancho_total_in - ancho_col0
    
    content_table = doc.add_table(rows=6, cols=2)
    content_table.autofit = False
    content_table.allow_autofit = False
    content_table.alignment = WD_TABLE_ALIGNMENT.CENTER
    content_table.preferred_width = Inches(ancho_total_in)
    
    # Configurar columnas
    for i, width in enumerate([ancho_col0, ancho_col1]):
        content_table.columns[i].width = Inches(width)
        content_table.columns[i].preferred_width = Inches(width)
    
    # Quitar bordes de la tabla
    borders_xml = r'<w:tcBorders xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">' \
                 r'<w:top w:val="none" w:sz="0" w:space="0" w:color="auto"/>' \
                 r'<w:left w:val="none" w:sz="0" w:space="0" w:color="auto"/>' \
                 r'<w:bottom w:val="none" w:sz="0" w:space="0" w:color="auto"/>' \
                 r'<w:right w:val="none" w:sz="0" w:space="0" w:color="auto"/>' \
                 r'</w:tcBorders>'
    
    for row in content_table.rows:
        for cell in row.cells:
            tcPr = cell._element.tcPr
            tcBorders = parse_xml(borders_xml)
            tcPr.append(tcBorders)
    
    # Llenar tabla con datos
    datos_tabla = [
        ("A:", "DIRECTOR MUNICIPAL DE SALUD GAMLP GOBIERNO AUTÓNOMO MUNICIPAL DE LA PAZ"),
        ("VÍA:", "LABORATORIO FARMACOLÓGICO"),
        ("DE:", "RESPONSABLE DE LA ELABORACIÓN DEL INFORME BIOINGENIERA ANABEL ROSSIO PLATA BURGOA"),
        ("REF.:", f"INFORME DE REVISIÓN DEL CUMPLIMIENTO DE INFORME GENÉTICO DE LA BACTERIA {bacteria_nombre} REALIZADA EN EL HOSPITAL MUNICIPAL DE SEGUNDO NIVEL LA PORTADA CORRESPONDIENTE AL MES DE JUNIO DE LA GESTIÓN 2025"),
        ("FECHA:", "9 de junio de 2025"),
        ("", "") 
    ]
    
    for i, (etiqueta, valor) in enumerate(datos_tabla):
        # Celda izquierda (etiqueta)
        cell_left = content_table.cell(i, 0).paragraphs[0]
        run_left = cell_left.add_run(etiqueta)
        run_left.bold = True
        run_left.font.name = 'Tahoma'
        run_left.font.size = Pt(10)
        
        # Celda derecha (valor)
        cell_right = content_table.cell(i, 1).paragraphs[0]
        run_right = cell_right.add_run(valor)
        run_right.bold = True if i != 4 else False  # Solo la fecha no en negrita
        run_right.font.name = 'Tahoma'
        run_right.font.size = Pt(10)
        
        # Alineación especial para fecha
        if i == 4:  # Fila de fecha
            cell_right.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Línea horizontal
    add_horizontal_line(doc)
    
    # Configuración de estilo general
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Tahoma'
    font.size = Pt(10)
    
    # Contenido del informe
    secciones = [
        ("De mi mayor consideración:\n\n", None, "left"),
        ("En cumplimiento a lo establecido en el Reglamento para la aplicación técnica y la gestión administrativa y financiera de la Ley N° 1152, aprobado mediante Resolución Ministerial N° 0251 de 30 de junio de 2021, remito el informe de revisión del cumplimiento de la Correlación Clínica Administrativa de los Servicios de Salud otorgados a la población beneficiaria, en el Hospital Municipal de Segundo Nivel La Portada la gestión 202X.", None),
        
        ("1. ANTECEDENTES", True),
        ("La Ley N° 475, de 30 de diciembre de 2013, de Prestaciones de Servicios de Salud Integral del Estado Plurinacional de Bolivia, tiene por objeto establecer y regular la atención integral y la protección financiera en salud de la población beneficiaria que no se encuentre cubierta por el Seguro Social Obligatorio de Corto Plazo y establecer las bases para la universalización de la atención integral en salud.", False),
        ("La Ley N° 1069, de 28 de mayo de 2018, modifica la Ley N° 475, para optimizar el uso de los recursos financieros asignados a la atención integral de salud.", False),
        ("La Ley N° 1152, de 20 de febrero de 2019, modifica la Ley N° 475, para ampliar la población beneficiaria que no se encuentra cubierta por la Seguridad Social de Corto Plazo, con atención gratuita en salud, en avance hacia un Sistema Único de Salud, Universal y Gratuito.", False),
        ("La Resolución Ministerial N° 0251, de 30 de junio de 2021, aprueba el Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de La Ley N° 1152 de 20 de febrero de 2019 'Hacia el Sistema Único de Salud Universal y Gratuito'.", False),
       
        ("2. JUSTIFICACIÓN", True),
        ("La correlación clínica administrativa, es la coherencia existente entre los diagnósticos y procedimientos efectuados por el personal de salud, registrados en el expediente clínico, que comprende la historia clínica, epicrisis, hojas de enfermería, hojas de evolución y otros pertinentes a la atención, con los servicios de salud declarados en los documentos administrativos generados por las herramientas informáticas del establecimiento, la misma que debe ser realizada con el propósito de mejorar la calidad del dato y la atención de los pacientes y retroalimentar al personal de salud y administrativo.", False),
        ("El Hospital La Portada, en cumplimiento al Parágrafo VI, inciso b) del Artículo 53 del Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152, que señala como uno de los documentos para el proceso de cobro intermunicipal a los GAM's o GAIOC's deudores, el informe de Correlación Clínica Administrativa de los pacientes referidos y en tránsito de dichos municipios.", False),
       
        ("3. METODOLOGÍA", True),
        ("El Informe de Correlación Clínica Administrativa, se elaboró bajo la siguiente metodología:", False),
        
        ("a) Se realizó la revisión de los expedientes clínicos de pacientes atendidos según incisos a) y b) del parágrafo I del artículo 53 del Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152, incluidos en los cobros intermunicipales y se los comparó con los registros del SIAF del establecimiento.", False),
        
        ("b) Para este fin se aplicó el instrumento de verificación proporcionado por el Ministerio de Salud y Deportes, que incluye las siguientes variables:", False),
        ("• Verificación de los datos generales de cada paciente.", False),
        ("• Identificación del Diagnóstico Principal.", False),
        ("• Identificación de posibles complicaciones y comorbilidades del paciente.", False),
        ("• Procedimientos terapéuticos y diagnósticos otorgados.", False),
        
        ("c) Identificación y descripción de las principales inconsistencias encontradas.", False),
        ("d) Descripción de las acciones correctivas realizadas en el establecimiento de salud.", False),
       
        ("4. ANÁLISIS TÉCNICO", True),
        ("Resultados", True),
        ("De la revisión de la muestra señalada de los expedientes clínicos y de los servicios de salud declarados en el SIAF del establecimiento de salud, se obtuvieron los siguientes resultados generales:", False),
        ("Acciones correctivas realizadas", True),
        ("Con los resultados de la revisión de la correlación clínica administrativa del hospital de Segundo Nivel La Portada, se realizaron las siguientes acciones correctivas por parte de la Dirección del establecimiento de salud, con insumos proporcionados por el responsable del SUS del Hospital:", False),
        ("       1. Se retirará del cobro intermunicipal al GAM o GAIOC deudor, aquellos expedientes clínicos que presentan observaciones.", False),
        
        ("5. CONCLUSIONES", True)
    ]
    
    for i, seccion in enumerate(secciones):
        p = doc.add_paragraph()
        p.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
        p.paragraph_format.space_after = Pt(6)
        
        if i == 0:
            texto, es_titulo, alineacion = seccion
            p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
            p.paragraph_format.first_line_indent = Pt(0)
            run = p.add_run(texto)
        else:
            texto, es_titulo = seccion[:2]  # Solo toma los primeros 2 elementos
            p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT if es_titulo else WD_PARAGRAPH_ALIGNMENT.JUSTIFY
            p.paragraph_format.first_line_indent = Inches(0.5) if es_titulo else Pt(0)
            run = p.add_run(texto)
            if es_titulo:
                run.bold = True
        
        run.font.name = 'Tahoma'
        run.font.size = Pt(10)
    
    # Añadir resultados específicos de los genes
    for n, gen, resultado in [
        (1, "blaTEM-1", resultado_blatem1),
        (2, "blaCTX-M-15", resultado_blactxm15)
    ]:
        # Descripción del análisis
        p_desc = doc.add_paragraph()
        p_desc.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
        p_desc.paragraph_format.space_after = Pt(6)
        p_desc.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
        add_run(p_desc, f"{n}. Se realizó la verificación de la posible resistencia de la bacteria ")
        add_run(p_desc, bacteria_nombre, highlight=True)
        add_run(p_desc, f". De la que se sospecha posee el gen de resistencia '{gen}', del análisis genético realizado se puede concluir que esta es:")
        
        # Resultado (POSITIVO/NEGATIVO)
        p_res = doc.add_paragraph()
        p_res.paragraph_format.space_after = Pt(12)
        p_res.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        add_run(p_res, "POSITIVO" if resultado else "NEGATIVO", bold=True, highlight=True)
    # Añadir la imagen (ajusta la ruta a tu imagen)
    ruta_imagen = r"C:\Tareas002\SegundoParcial\Firma.eje1.jpg"  
    
    try:
        # Añadir imagen centrada
        paragraph_img = doc.add_paragraph()
        paragraph_img.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        paragraph_img.paragraph_format.space_after = Inches(0)  
        run_img = paragraph_img.add_run()
        run_img.add_picture(ruta_imagen, width=Inches(2.0))  
      
        
    except FileNotFoundError:
        print(f"Advertencia: No se encontró la imagen en {ruta_imagen}")
        doc.add_paragraph("[Firma o imagen no disponible]", style='Intense Quote')
    
    # Firma 
    p_nombre = doc.add_paragraph()
    p_nombre.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    p_nombre.paragraph_format.space_after = Pt(0)
    add_run(p_nombre, "INGENIERA ANABEL ROSSIO PLATA BURGOA", bold=True)
    
    p_ci = doc.add_paragraph()
    p_ci.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    p_ci.paragraph_format.space_after = Pt(12)
    add_run(p_ci, "CI: 12830436 LP")

    # Crear directorio de salida si no existe
    os.makedirs(salida_docx, exist_ok=True)
    
    # Guardar documento
    nombre_salida = os.path.join(salida_docx, f"informe_{bacteria_nombre}.docx")
    doc.save(nombre_salida)
    print(f"Informe generado: {nombre_salida}")

def generar_resumen_excel(resultados, ruta_salida):
    
    # Convertir a DataFrame
    df = pd.DataFrame(resultados)
    
    # Ordenar columnas
    columnas = ['Muestra', 'blaTEM-1', 'blaCTX-M-15']
    df = df[columnas]
    
    # Guardar en Excel
    df.to_excel(ruta_salida, index=False, engine='openpyxl')
    print(f"\nResumen de resultados guardado en: {ruta_salida}")

# === VERIFICACIÓN DE RUTAS ===
if not os.path.exists(carpeta_fna):
    raise FileNotFoundError(f"No se encuentra la carpeta de genomas: {carpeta_fna}")

for archivo in [archivo_blatem1, archivo_blactxm15]:
    if not os.path.exists(archivo):
        raise FileNotFoundError(f"No se encuentra el archivo de gen: {archivo}")

# directorio de salida 
os.makedirs(salida_docx, exist_ok=True)

# === CARGA DE GENES ===
gen_blatem1 = leer_gen(archivo_blatem1)
gen_blactxm15 = leer_gen(archivo_blactxm15)

# Mostrar información básica de los genes de referencia
print(f"\nGenes de referencia cargados:")
print(f"- blaTEM-1: {len(gen_blatem1)} bp")
print(f"- blaCTX-M-15: {len(gen_blactxm15)} bp")

# === PROCESAMIENTO DE GENOMAS ===
print("\nIniciando análisis de genomas...")

# Lista para almacenar todos los resultados
resultados_totales = []

for archivo in os.listdir(carpeta_fna):
    if archivo.endswith(".fna"):
        ruta_completa = os.path.join(carpeta_fna, archivo)
        nombre_muestra = archivo.replace(".fna", "")
        
        try:
            # Procesar secuencias
            secuencias = "".join(str(record.seq).upper() for record in SeqIO.parse(ruta_completa, "fasta"))
            
            # Búsqueda en ambas hebras
            gen_blatem1_rc = reverse_complement(gen_blatem1)
            gen_blactxm15_rc = reverse_complement(gen_blactxm15)
            
            tiene_blatem1 = (gen_blatem1 in secuencias) or (gen_blatem1_rc in secuencias)
            tiene_blactxm15 = (gen_blactxm15 in secuencias) or (gen_blactxm15_rc in secuencias)

            # Mostrar resultados en consola
            print(f"\nResultados para {archivo}:")
            print(f"  {'✓' if tiene_blatem1 else '✗'} Gen blaTEM-1 {'detectado' if tiene_blatem1 else 'no detectado'}")
            print(f"  {'✓' if tiene_blactxm15 else '✗'} Gen blaCTX-M-15 {'detectado' if tiene_blactxm15 else 'no detectado'}")

            # Generar informe DOCX
            generar_informe(archivo, tiene_blatem1, tiene_blactxm15)
            
            # Guardar resultados para el Excel
            resultados_totales.append({
                'Muestra': nombre_muestra,
                'blaTEM-1': 'POSITIVO' if tiene_blatem1 else 'NEGATIVO',
                'blaCTX-M-15': 'POSITIVO' if tiene_blactxm15 else 'NEGATIVO'
            })
            
        except Exception as e:
            print(f"Error al procesar {archivo}: {str(e)}")
            continue

# Generar resumen en Excel
if resultados_totales:
    generar_resumen_excel(resultados_totales, salida_excel)

print("\nAnálisis completado para todos los genomas.")


Genes de referencia cargados:
- blaTEM-1: 979 bp
- blaCTX-M-15: 2315 bp

Iniciando análisis de genomas...

Resultados para GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna:
  ✗ Gen blaTEM-1 no detectado
  ✗ Gen blaCTX-M-15 no detectado
Informe generado: C:\Tareas002\SegundoParcial\Informes\informe_GCF_050389795.1_MS10495_ARL02201_ST410_genomic.docx

Resultados para GCF_050516345.1_ASM5051634v1_genomic.fna:
  ✗ Gen blaTEM-1 no detectado
  ✗ Gen blaCTX-M-15 no detectado
Informe generado: C:\Tareas002\SegundoParcial\Informes\informe_GCF_050516345.1_ASM5051634v1_genomic.docx

Resultados para GCF_050516355.1_ASM5051635v1_genomic.fna:
  ✗ Gen blaTEM-1 no detectado
  ✗ Gen blaCTX-M-15 no detectado
Informe generado: C:\Tareas002\SegundoParcial\Informes\informe_GCF_050516355.1_ASM5051635v1_genomic.docx

Resultados para GCF_050516365.1_ASM5051636v1_genomic.fna:
  ✗ Gen blaTEM-1 no detectado
  ✗ Gen blaCTX-M-15 no detectado
Informe generado: C:\Tareas002\SegundoParcial\Informes\informe_GCF_050